# 0. Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

## 5. Data Modeling
**Bài toán**: Dự đoán một công ty đã nằm trong bảng xếp hạng FT1000 có tiếp tục nằm trong bảng xếp hạng FT1000 năm sau hay không?

**Ý nghĩa**: Cùng ý nghĩa với các câu hỏi ở phần Data Analyzing, khi biết được một công ty đã được xếp hạng trong FT1000 năm nay có tiếp tục nằm trong bảng xếp hạng FT1000 vào năm sau hay không giúp các nhà đầu tư đưa ra lựa chọn đầu tư chính xác hơn, bên cạnh các chỉ số về doanh thu hay mức tăng trưởng đã được phân tích trong bảng xếp hạng năm nay

## 5.1. Thu thập thêm dữ liệu
Dữ liệu bảng xếp hạng năm nay là không đủ để  giải quyết bài toán, ta cần thu thập dữ liệu của cả những bảng xếp hạng trước. Bảng xếp hạng FT tính đến namư 2022 được tổ chức lần thứ 6, do đó ta cần thu thập thêm dữ liệu từ 5 bảng xếp hạng ở 5 năm trước. Phương pháp thu thập cũng sẽ giống việc thu thập ở phần Data Collecting

In [2]:
url = 'https://ig.ft.com/ft-1000/'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2017 = pd.DataFrame(js[0])
df2017.rename(columns={'Unnamed: 0':'Rank'}, inplace = True)
df2017 = df2017.sort_values(by = 'Rank').reset_index(drop = True)
df2017.head()

,Rank,Company,Country,Sector,RevenueGrowth 2012-5,CAGR2012-5,Revenue2015 (’000 €),Employees2015,Founded
0,1,HelloFresh,Germany,Food & Beverage,"13,159%",409.9%,304952,981.0,2011
1,2,Codewise,Poland,Advertising,"13,052%",408.5%,31613,103.0,2011
2,3,Green IT Das Systemhaus,Germany,Technology,"11,113%",382.2%,24305,80.0,2012
3,4,Brainlabs,United Kingdom,Advertising,"8,218%",336.5%,13608,48.0,2012
4,5,Catapult Enterprises (Propercorn),United Kingdom,Food & Beverage,"7,510%",323.8%,12052,32.0,2011


In [3]:
url = 'https://www.ft.com/content/cf0c5fce-3112-11e8-b5bf-23cb17fd1498'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2018 = pd.DataFrame(js[0])
df2018.head()

,Rank,Company,Location,Sector,Revenue growth 2013-6 (%),CAGR 2013-6 (%),Revenue 2016 (€'000),Employee growth 2013-6 (%),Employees 2016,Founded
0,1,Deliveroo*,United Kingdom,Food & Beverage,107117,923.5,157406,1046.0,1049.0,2013
1,2,Thermondo,Germany,Energy,10878,378.8,20222,280.0,300.0,2012
2,3,Traventia Viajes*,Spain,Travel & Leisure,9377,355.9,10295,19.0,20.0,2013
3,4,Alainsa,Spain,Construction,9114,351.7,10286,19.0,20.0,1980
4,5,iTravex,Spain,Travel & Leisure,8398,339.6,22674,28.0,37.0,2007


In [4]:
url = 'https://www.ft.com/content/238174d2-3139-11e9-8744-e7016697f225'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2019 = pd.DataFrame(js[0])
df2019.head()

,Rank,Company,In 2018 list?,In 2017 list?,Country,Sector,Absolute revenue growth,Revenue CAGR 2014-17,Revenue 2017 (€m),Employee growth 2014-17,Employees 2017,Founded
0,1,Blue Motor Finance,yes,no,United Kingdom,Financial Services,"51,364%",701.4%,61.4,143.0,155.0,1992
1,2,Deliveroo,yes,no,United Kingdom,Food & Beverage,"15,749%",441.2%,316.1,1636.0,1664.0,2013
2,3,Taxify,no,no,Estonia,Technology,"12,231%",397.7%,17.8,336.0,350.0,2013
3,4,Solectric,no,no,Germany,Technology,"7,772%",328.6%,40.9,27.0,30.0,2012
4,5,Psioxus Therapeutics,no,no,United Kingdom,Pharmaceuticals,"7,126%",316.5%,64.4,41.0,69.0,2006


In [5]:
url = 'https://www.ft.com/content/691390ca-53d9-11ea-90ad-25e377c0ee1f'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2020 = pd.DataFrame(js[0])
df2020.head()

,Rank,Name,in 2019 ranking,in 2018 ranking,Country,FT Category,Absolute Growth Rate [in %],Compound Annual Growth Rate (CAGR) [in %],Revenue 2018 [in €m],Revenue 2015 [in €m],Number of employees 2018,Founding Year
0,1,OakNorth Bank,no,no,United Kingdom,Fintech,"37,462.5%",621.5%,67.91,0.22,200.0,2015
1,2,Wolt Enterprises,no,no,Finland,Technology,"15,641.9%",439.9%,30.07,0.19,293.0,2014
2,3,Bolt Technology,yes,no,Estonia,Technology,"12,959.7%",407.4%,79.68,0.61,700.0,2013
3,4,Elements Global Services,no,no,Spain,Support Services,"10,233.3%",369.3%,26.25,0.27,38.0,2015
4,5,Les Eco-Isolateurs,yes,no,France,Construction,"10,077.5%",366.9%,32.98,0.32,67.0,2013


In [6]:
url = 'https://www.ft.com/content/8b37a92b-15e6-4b9c-8427-315a8b5f4332'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
js = pd.read_html(str(soup.find_all('table')))

df2021 = pd.DataFrame(js[0])
df2021.iloc[:, :13].head()

,Rank,Name,in 2020 ranking,in 2019 ranking,Country,FT Category,Absolute Growth Rate %,Compound Annual Growth Rate (CAGR) %,Revenue 2019 €,Revenue 2016 €,Number of employees 2019,Number of employees 2016,Founding Year
0,1,Bulb Energy,No,No,UK,Energy,199626.7,1159.3,1737586657,985867,575,55,2015
1,2,Sun Finance,No,No,Latvia,Fintech,61837.8,752.4,101475796,163835,750,3,2012
2,3,Everflow Group,No,No,UK,Energy,17940.4,465.0,55902990,363266,46,4,2015
3,4,Glencar Construction,No,No,UK,Construction,15735.4,441.0,110298923,724551,85,2,2015
4,5,Spyrosoft,No,No,Poland,Technology,14783.8,430.0,16635991,110092,376,12,2016


In [7]:
df2022 = pd.read_csv('data/FT1000_v1.csv')
df2022.head()

,Rank,Name,Ranked2021,Ranked2020,Country,Sector,CAGR,Revenue2020,Revenue2017,Employees2020,Employees2017,FoundingYear,Country1,Sector1
0,1,Swappie,No,No,Finland,Technology,477.43,97611814,507000,218,1,2016,Finland,Technology
1,2,Kilo Health,No,No,Lithuania,Health,450.05,57318766,344428,177,10,2013,Lithuania,Health
2,3,OCI,No,No,UK,Financial Services,409.59,568322073,4325512,32,4,2012,UK,Financial Services
3,4,OnlyFans,No,No,UK,Technology,393.63,316732986,2652185,800,4,2016,UK,Technology
4,5,Enpal,No,No,Germany,Energy,386.88,56109613,486165,365,9,2017,Germany,Energy


Quan sát các bảng xếp hạng các năm trước, nhóm nhận thấy đặc điểm về các cột như sau:
- Các cột bảng nào cũng có:
    - Rank
    - Name
    - Country (ở bảng 2018 tên là Location)
    - Sector (ở bảng 2020 và 2021 là FT Category)
    - Absolute Growth Rate (có bảng có đơn vị %, có bảng chỉ có số thực)
    - CAGR (tương tự AGR bên trên)
    - Doanh thu năm cuối trong chu kì 3 năm (mỗi bảng lại có một dơn vị khác nhau)
    - Số lượng nhân viên năm cuối trong chu kì 3 năm 
    - FoundingYear - Năm thành lập (ở một số bảng là Founded)
- Các cột chỉ một vài bảng có:
    - Số nhân viên năm đầu tiên trong chu kì 3 năm (chỉ bảng 2021)
    - Doanh thu năm đầu tiên trong chu kì 3 năm (chỉ bảng 2020, 2021)
    - Mức tăng trưởng nhân viên trong 3 năm (chỉ bảng 2018, 2019)
    - Có mặt trong bảng xếp hạng 2 năm trước (chỉ bảng 2019, 2020, 2021)

- Do đó nhóm quyết định sẽ có 2 bảng sample dùng để dự đoán, có chung các trường sau:
    - Rank
    - Country
    - Sector    
    - CAGR
    - Doanh thu năm cuối cùng trong chu kì 3 năm
    - Số lượng nhân viên năm cuối cùng trong chu kì 3 năm
    - FoundingYear
    
- Về lí do có 2 bảng sample là vì năm 2017 và 2018, ta không có các trường xác định công ty có nằm trong bảng xếp hạng 2 năm liền trước hay không, do đó dữ liệu chia làm 2 nhóm:
    - Nhóm 1: Gồm bảng 2019, 2020, 2021: Nhóm biết công ty có nằm trong bảng 2 năm liền trước
    - Nhóm 2: Gồm bảng 2018, 2019, 2020, 2021: Nhóm biết công ty có nằm trong bảng 1 năm liền trước. Tuy bảng 2018 không có trường này nhưng ta có thể dễ dàng xác định được thông qua bảng 2017

Do đó ta sẽ có 2 loại sample để dự đoán công ty có nằm trong bảng xếp hạng năm sau hay không, đồng nghĩa với 2 mô hình học máy. Việc gán nhãn này cũng được thực hiện dễ dàng bằng cách dựa vào các bảng có sẵn.

Bên dưới đây là các thao tác làm sạch dữ liệu ở từng bảng trước ghi gom các bảng thành 1 bảng chung. Các thao tác này tương tự với phần đã trình bày ở Data Preprocessing nên sẽ không trình bày thêm ở đây.

## 5.2. Tiền xử lí

In [8]:
df2018.rename(columns = {
    'Company': 'Name',
    'Location': 'Country',
    'CAGR 2013-6 (%)': 'CAGR',
    "Revenue 2016 (€'000)": 'Revenue',
    'Employees 2016': 'Employees',
    'Founded': 'FoundingYear'
}, inplace = True)

df2018['Last'] = df2018['Name'].isin(df2017['Company'])
df2018['Next'] = df2018['Name'].isin(df2019['Company'])
df2018.replace({True: 1, False: 0}, inplace = True)
df2018['Revenue'] = df2018['Revenue'] / 1000
df2018['Employees'] = df2018['Employees'].fillna(df2018['Employees'].median()).astype(int)
df2018['FoundingYear'] = df2018['FoundingYear'].astype(int)
df2018.drop(columns = ['Name', 'Revenue growth 2013-6 (%)', 'Employee growth 2013-6 (%)'], inplace = True)

df2018

,Rank,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Last,Next
0,1,United Kingdom,Food & Beverage,923.5,157.406,1049,2013,0,0
1,2,Germany,Energy,378.8,20.222,300,2012,0,0
2,3,Spain,Travel & Leisure,355.9,10.295,20,2013,0,0
3,4,Spain,Construction,351.7,10.286,20,1980,0,0
4,5,Spain,Travel & Leisure,339.6,22.674,37,2007,0,0
...,...,...,...,...,...,...,...,...,...
995,996,Italy,Chemicals,34.7,4.043,5,1989,0,0
996,997,Austria,Support Services,34.7,2.075,6,2013,0,0
997,998,Germany,Advertising,34.7,7.011,30,2003,0,1
998,999,Spain,Restaurants,34.6,3.100,87,1997,1,0


In [9]:
df2019.rename(columns = {
    'Company': 'Name',
    'In 2018 list?': 'Last',
    'In 2017 list?': '2Last',
    'Revenue CAGR 2014-17': 'CAGR',
    'Revenue 2017 (€m)': 'Revenue',
    'Employees 2017': 'Employees',
    'Founded': 'FoundingYear'
}, inplace = True)

df2019['Next'] = df2019['Name'].isin(df2020['Name'])
df2019['CAGR'] = df2019['CAGR'].str.replace('%', '').astype(float)
df2019.replace({'yes': 1, 'no': 0}, inplace = True)
df2019.replace({True: 1, False: 0}, inplace = True)
df2019['Employees'] = df2019['Employees'].fillna(df2019['Employees'].median()).astype(int)
df2019['FoundingYear'] = df2019['FoundingYear'].astype(int)
df2019.drop(columns = ['Name', 'Absolute revenue growth', 'Employee growth 2014-17'], inplace = True)

df2019

,Rank,Last,2Last,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Next
0,1,1,0,United Kingdom,Financial Services,701.4,61.4,155,1992,1
1,2,1,0,United Kingdom,Food & Beverage,441.2,316.1,1664,2013,0
2,3,0,0,Estonia,Technology,397.7,17.8,350,2013,0
3,4,0,0,Germany,Technology,328.6,40.9,30,2012,0
4,5,0,0,United Kingdom,Pharmaceuticals,316.5,64.4,69,2006,0
...,...,...,...,...,...,...,...,...,...,...
995,996,1,0,France,Management Consulting,37.8,2.2,12,2011,0
996,997,0,0,United Kingdom,Health,37.8,21.4,108,2010,0
997,998,0,0,Germany,Retail,37.8,11.1,15,2004,0
998,999,1,0,Germany,Fashion,37.8,3.3,19,2007,0


In [10]:
df2020.rename(columns = {
    'in 2019 ranking': 'Last',
    'in 2018 ranking': '2Last',
    'FT Category': 'Sector',
    'Compound Annual Growth Rate (CAGR) [in %]': 'CAGR',
    'Revenue 2018 [in €m]': 'Revenue',
    'Number of employees 2018': 'Employees',
    'Founding Year': 'FoundingYear'
}, inplace = True)

df2020['Next'] = df2020['Name'].isin(df2021['Name'])
df2020['CAGR'] = df2020['CAGR'].str.replace('%', '').astype(float)
df2020.replace({'yes': 1, 'no': 0}, inplace = True)
df2020.replace({True: 1, False: 0}, inplace = True)
df2020['Employees'] = df2020['Employees'].fillna(df2020['Employees'].median()).astype(int)
df2020['FoundingYear'] = df2020['FoundingYear'].astype(int)
df2020.drop(columns = ['Name', 'Absolute Growth Rate [in %]', 'Revenue 2015 [in €m]'], inplace = True)

df2020

,Rank,Last,2Last,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Next
0,1,0,0,United Kingdom,Fintech,621.5,67.91,200,2015,1
1,2,0,0,Finland,Technology,439.9,30.07,293,2014,0
2,3,1,0,Estonia,Technology,407.4,79.68,700,2013,1
3,4,0,0,Spain,Support Services,369.3,26.25,38,2015,0
4,5,1,0,France,Construction,366.9,32.98,67,2013,0
...,...,...,...,...,...,...,...,...,...,...
995,996,0,0,France,Energy,38.5,2.23,36,2009,1
996,997,0,0,Spain,Construction,38.5,11.06,6,2012,0
997,998,1,0,Italy,Support Services,38.5,4.65,9,2009,0
998,999,1,0,Italy,Industrial Goods,38.4,2.26,6,2002,0


In [11]:
df2021 = df2021.iloc[:1000, :13]

df2021.rename(columns = {
    'in 2020 ranking': 'Last',
    'in 2019 ranking': '2Last',
    'FT Category': 'Sector',
    'Compound Annual Growth Rate (CAGR) %': 'CAGR',
    'Revenue 2019 €': 'Revenue',
    'Number of employees 2019': 'Employees',
    'Founding Year': 'FoundingYear'
}, inplace = True)

df2021['Next'] = df2021['Name'].isin(df2022['Name'])
df2021.replace({'Yes': 1, 'No': 0}, inplace = True)
df2021.replace({True: 1, False: 0}, inplace = True)
df2021['Revenue'] = df2021['Revenue'].astype(float) / 1000000
df2021['Employees'] = df2021['Employees'].fillna(df2021['Employees'].median()).astype(int)
df2021['FoundingYear'] = df2021['FoundingYear'].astype(int)
df2021['CAGR'] = df2021['CAGR'].astype(float)
df2021['FoundingYear'].astype(int)
df2021['Employees'].astype(int)
df2021['Rank'] = df2021['Rank'].astype(int)
df2021.drop(columns = ['Name', 'Absolute Growth Rate %', 'Revenue 2016 €', 'Number of employees 2016'], inplace = True)

df2021

,Rank,Last,2Last,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Next
0,1,0,0,UK,Energy,1159.3,1737.586657,575,2015,0
1,2,0,0,Latvia,Fintech,752.4,101.475796,750,2012,1
2,3,0,0,UK,Energy,465.0,55.902990,46,2015,0
3,4,0,0,UK,Construction,441.0,110.298923,85,2015,1
4,5,0,0,Poland,Technology,430.0,16.635991,376,2016,1
...,...,...,...,...,...,...,...,...,...,...
995,996,1,0,Italy,Advertising,35.7,1.674159,19,2011,0
996,997,0,0,France,Energy,35.6,2.370118,10,2005,0
997,998,1,0,France,Energy,35.6,2.539125,32,2009,0
998,999,0,0,Germany,Chemicals,35.5,2.567360,7,2000,0


## 5.3. Kiểm tra kiểu dữ liệu và missing value

In [12]:
df2018.isna().sum()

Rank            0
Country         0
Sector          0
CAGR            0
Revenue         0
Employees       0
FoundingYear    0
Last            0
Next            0
dtype: int64

In [13]:
df2019.isna().sum()

Rank            0
Last            0
2Last           0
Country         0
Sector          0
CAGR            0
Revenue         0
Employees       0
FoundingYear    0
Next            0
dtype: int64

In [14]:
df2020.isna().sum()

Rank            0
Last            0
2Last           0
Country         0
Sector          0
CAGR            0
Revenue         0
Employees       0
FoundingYear    0
Next            0
dtype: int64

In [15]:
df2021.isna().sum()

Rank            0
Last            0
2Last           0
Country         0
Sector          0
CAGR            0
Revenue         0
Employees       0
FoundingYear    0
Next            0
dtype: int64

In [16]:
def open_object_dtype(s):
    dtypes = s.apply(lambda item : type(item))
    return dtypes.unique()

df2018.apply(open_object_dtype)

,Rank,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Last,Next
0,<class 'int'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>,<class 'int'>


In [17]:
df2019.apply(open_object_dtype)


,Rank,Last,2Last,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Next
0,<class 'int'>,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>


In [18]:
df2020.apply(open_object_dtype)


,Rank,Last,2Last,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Next
0,<class 'int'>,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>


In [19]:
df2021.apply(open_object_dtype)


,Rank,Last,2Last,Country,Sector,CAGR,Revenue,Employees,FoundingYear,Next
0,<class 'int'>,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'int'>


## 5.4 Gộp các bảng xếp hạng lại thành sample

In [20]:
df1 = pd.concat([df2019, df2020, df2021])

In [21]:
df2_feature = ['Rank','Last', 'Country', 'Sector', 'CAGR', 'Revenue', 'Employees', 'FoundingYear', 'Next']
df2 = pd.concat([df2018, df2019[df2_feature], df2020[df2_feature], df2021[df2_feature]])

In [22]:
X1 = np.array(pd.get_dummies(df1.iloc[:, :-1]))
X2 = np.array(pd.get_dummies(df2.iloc[:, :-1]))
Y1 = np.array(df1.iloc[:, -1])
Y2 = np.array(df2.iloc[:, -1])